In [2]:
import sys
sys.path.append('../')

In [3]:
from DPF import ShardsDatasetConfig, DatasetReader

config = ShardsDatasetConfig.from_path_and_columns(
    'example_dataset',
    image_name_col='image_name',
    text_col="caption"
)

reader = DatasetReader()
processor = reader.read_from_config(config)

  0%|          | 0/3 [00:00<?, ?it/s]

# Text filters

## language detection

In [4]:
from DPF.filters.texts.lang_filter import LangFilter

column_filter = LangFilter(
    text_column_name='text',
)
print('Columns will be added:', column_filter.result_columns)

processor.apply_column_filter(column_filter)

Columns will be added: ['lang', 'lang_score']
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
processor.df

,image_path,text,split_name,lang,lang_score
0,example_dataset/0.tar/0.jpg,шотландцы в национальной одежде с флагом. - ba...,0,ru,1.0
1,example_dataset/0.tar/1.jpg,королевская золотая корона с драгоценностями н...,0,ru,1.0
2,example_dataset/0.tar/2.jpg,Пасха,0,be,0.49
3,example_dataset/0.tar/3.jpg,округа нью-джерси карта печати - elizabeth sto...,0,uk,1.0
4,example_dataset/0.tar/4.jpg,"английская золотая корона с драгоценностями, и...",0,ru,1.0
...,...,...,...,...,...
495,example_dataset/2.tar/495.jpg,bokeh Background,2,de,0.35
496,example_dataset/2.tar/496.jpg,"Healthcare, Medicine,",2,ro,0.96
497,example_dataset/2.tar/497.jpg,A set of business teams icons that include edi...,2,en,1.0
498,example_dataset/2.tar/498.jpg,Vector floral pattern icon collection,2,es,0.59


## Google translate

In [6]:
from DPF.filters.texts.google_translate_filter import GoogleTranslateFilter

column_filter = GoogleTranslateFilter(
    text_column_name='text',
    max_symbols_in_batch=2000,
    timeout_on_error=10,
    num_retries_per_batch=2
)
print(column_filter.result_columns)

processor.apply_column_filter(column_filter)

['text_translated']


 33%|███▎      | 9/27 [00:09<00:13,  1.33it/s]

[GoogleTranslateFilter] , retry: 0/2
[GoogleTranslateFilter] , retry: 1/2
[GoogleTranslateFilter] , retry: 2/2


 59%|█████▉    | 16/27 [00:41<00:15,  1.42s/it]

[GoogleTranslateFilter] , retry: 0/2
[GoogleTranslateFilter] , retry: 1/2
[GoogleTranslateFilter] , retry: 2/2


 67%|██████▋   | 18/27 [01:12<01:04,  7.16s/it]

[GoogleTranslateFilter] , retry: 0/2
[GoogleTranslateFilter] , retry: 1/2
[GoogleTranslateFilter] , retry: 2/2


100%|██████████| 27/27 [01:47<00:00,  3.97s/it]


In [7]:
processor.df[['text', 'text_translated']]

,image_path,text,split_name,lang,lang_score,text_translated
0,example_dataset/0.tar/0.jpg,шотландцы в национальной одежде с флагом. - ba...,0,ru,1.0,Scots in national clothes with flag. - balmora...
1,example_dataset/0.tar/1.jpg,королевская золотая корона с драгоценностями н...,0,ru,1.0,королевская золотая корона с драгоценностями н...
2,example_dataset/0.tar/2.jpg,Пасха,0,be,0.49,Easter
3,example_dataset/0.tar/3.jpg,округа нью-джерси карта печати - elizabeth sto...,0,uk,1.0,New Jersey counties map print - elizabeth stoc...
4,example_dataset/0.tar/4.jpg,"английская золотая корона с драгоценностями, и...",0,ru,1.0,"английская золотая корона с драгоценностями, и..."
...,...,...,...,...,...,...
495,example_dataset/2.tar/495.jpg,bokeh Background,2,de,0.35,bokeh background
496,example_dataset/2.tar/496.jpg,"Healthcare, Medicine,",2,ro,0.96,"Healthcare, Medicine,"
497,example_dataset/2.tar/497.jpg,A set of business teams icons that include edi...,2,en,1.0,A set of business teams icons that include edi...
498,example_dataset/2.tar/498.jpg,Vector floral pattern icon collection,2,es,0.59,None
